# Lunaモデルの学習

### Lunaサイト
https://luna16.grand-challenge.org/Description/

### データセット概要
アーカイブを展開した状態のすべてのデータで約220GBを必要とする.  
+ annotations.csv
+ candidates.csv
+ sampleSubmission.csv
+ candidates_V2.zip
+ evaluationScript.zip
+ seg-lungs-LUNA16.zip
1. subset0.zip
2. subset1.zip
3. subset2.zip
4. subset3.zip
5. subset4.zip
6. subset5.zip
7. subset6.zip
8. subset7.zip
9. subset8.zip
10. subset10.zip

### subsetの中身
+ series_uidをユニークIDとして*.mhdと*.rawの2種類のファイルがある

In [1]:

import datetime
from p2_ct_project.util.logconf import logging
log = logging.getLogger('nb')
# log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

from p2_ct_project.util.util import importstr

# run_everything
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=8')
    # 4コア8スレッドのCPUを仮定
    log.info("Running: {}({!r}).main()".format(app, argv))
    print("*app.rsplit('.', 1) : ", *app.rsplit('.', 1))

    app_cls = importstr(*app.rsplit('.', 1)) # __import__を実行
    app_cls(argv).main()

    log.info("Finished: {}.({!r}).main()".format(app, argv))

In [2]:
import os
import shutil

# clean up any old data that might be around.
# We don't call this by default because it's destructive, 
# and would waste a lot of time if it ran when nothing 
# on the application side had changed.
tag_version = "unversioned"
cache_dir = f"F:/Luna16/cache/{tag_version}"
def cleanCache():
    shutil.rmtree(cache_dir)
    os.mkdir(cache_dir)

# cleanCache()

Fancacheのclose

In [3]:
from p2_ct_project.p2ch10.dataset_luna import raw_cache
raw_cache.close()

from p2_ct_project.p2ch12.mod_dataset_luna import raw_cache
raw_cache.close()

from p2_ct_project.p2ch13.dataset_luna_seg import raw_cache
raw_cache.close()

module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch10\..


c:\Users\InoueShinichi\anaconda3\envs\Py39WorkingEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch12\..
module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch13\..


### 各チャプターで共通の学習設定

In [4]:
experiment_epochs = 2
training_epochs = 20
final_epochs = 50
seg_epochs = 10

# Chapter11

### デフォルト設定で学習

In [5]:
# run('p2_ct_project.p2ch11.prepcache_luna.LunaPrepCacheApp', '--datasetdir=E:/Luna16')

In [6]:
# run('p2_ct_project.p2ch11.training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', '--epochs=1')

# Chapter12
+ 陽性サンプルと陰性サンプルの均一化

## 結節データをかさ増しする
+ LunaDatasetの結節候補データは, 結節でないデータと結節データの比率が均等でない

### 課題
+ ラベルが均等でないデータセットでは, 学習時にデータ比率が大きいラベル側にNNモデルが引っ張られ, 推論でデータ比率の大きなラベルばかりを推論結果として出力してしまう.

### 解決策
1. 学習時は, 各ラベルのデータ数を均等にする. 
2. 損失関数で, 比率の小さいラベルデータに対して更新勾配(誤差)を大きくする処理を追加する

#### 今回は比率の小さいラベルデータをかさ増しする

### 結節データ(比率の小さいラベルデータ)のかさ増しをする方法
1. PytorchのDataLoader関数の`sampler=...`オプションを使用する方法. <br> データセットの元々のイテレーション順序を上書きすることができ, <br> 特定のサンプルの抽出を制限したり, 逆に複数回抽出することが可能. これはデータセット自体をコントロールできない場合に非常に有効.
2. 直接データセットをコントロールできる場合, Datasetクラスのサブクラスの中で, 必要となるデータを再構成して, かさ増しする.

### データセット内でクラスバランスを調整する
+ LunaDataset内に追加の実装を行い, 訓練セットの陽性サンプル数と陰性サンプル数が1対1になるように変更する

In [7]:
# run('p2_ct_project.p2ch12.prepcache_luna.LunaPrepCacheApp', '--datasetdir=E:/Luna16')

In [8]:
# run('p2_ct_project.p2ch12.mod_training_luna.LunatrainingApp', '--datasetdir=E:/Luna16', '--epochs=1', 'unbalanced')

In [9]:
# データサンプルの調整
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', f'--epochs=1', '--balanced', 'balanced')

In [10]:
# import numpy as np
# a = np.empty((1, 100), dtype=np.float32) > 0.5
# print(a)
# ans = np.sum(a)
# print(ans)

### データ拡張の結果を比較する

In [11]:
# データサンプルの調整
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', f'--epochs={training_epochs}', '--balanced', 'balanced')

In [12]:
# データ拡張　フリップ
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-flip', 'flip')

In [13]:
# データ拡張 平行移動
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-offset', 'offset')

In [14]:
# データ拡張 スケール
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-scale', 'scale')

In [15]:
# データ拡張　回転
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-rotate', 'rotate')

In [16]:
# データ拡張　ノイズ付加
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-noise', 'noise')

In [17]:
# データ拡張　フリップ, 平行移動, スケール, 回転, ノイズ付加
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augmented', 'fully-augmented')

# Chapter13

セマンティックセグメンテーションモデルによる結節候補の推論

In [18]:
# 加工データセットのキャッシュを作成
# run('p2_ct_project.p2ch13.prepcache_luna_seg.LunaSegPrepCacheApp')

In [19]:
# datasetdir = 'E:/Luna16'
# run('p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp', 
#     f'--epochs={experiment_epochs}', 
#     '--augmented', 
#     f'--datasetdir={datasetdir}'
#     )

In [20]:
# datasetdir = 'E:/Luna16'
# seg_epochs = 10
# run('p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp', 
#     f'--epochs={seg_epochs}', 
#     '--augmented', 
#     'final-seg', 
#     f'--datasetdir={datasetdir}'
#     )

# Chapter14

EndtoEnd推論システムの統合

In [21]:
datasetdir = 'E:/Luna16'
run('p2_ct_project.p2ch14.prepcache_luna_end_to_end.LunaPrepCacheApp',
    f'--datasetdir={datasetdir}',
    )

2023-06-19 21:09:10,421 INFO     pid:18412 nb:015:run Running: p2_ct_project.p2ch14.prepcache_luna_end_to_end.LunaPrepCacheApp(['--num-workers=8', '--datasetdir=E:/Luna16']).main()


*app.rsplit('.', 1) :  p2_ct_project.p2ch14.prepcache_luna_end_to_end LunaPrepCacheApp
module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch14\..
module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch10\..\p2ch14\..


2023-06-19 21:09:10,821 INFO     pid:18412 p2_ct_project.p2ch14.prepcache_luna_end_to_end:060:main Starting LunaPrepCacheApp, Namespace(batch_size=1024, num_workers=8, datasetdir='E:/Luna16')


module_parent_dir c:\Users\InoueShinichi\Desktop\MyGithub\Book_HowToPytorch\p2_ct_project\p2ch10\..\p2ch11\..
module:  <module 'p2_ct_project' (namespace)>


2023-06-19 21:09:14,874 INFO     pid:18412 p2ch14.dataset_luna_end_to_end:449:__init__ <p2ch14.dataset_luna_end_to_end.LunaDataset object at 0x0000028B387A58E0>: 550896 training samples, 549714 neg, 1182 pos, unbalanced ratio
2023-06-19 21:09:14,875 WARNING  pid:18412 util.util:226:enumerateWithEstimate Stuffing cache ----/538, starting
2023-06-19 21:11:30,166 INFO     pid:18412 util.util:243:enumerateWithEstimate Stuffing cache   64/538, done at 2023-06-19 21:24:21, 0:14:24
2023-06-19 21:16:46,685 INFO     pid:18412 util.util:243:enumerateWithEstimate Stuffing cache  256/538, done at 2023-06-19 21:24:28, 0:14:31
2023-06-19 21:30:19,315 WARNING  pid:18412 util.util:256:enumerateWithEstimate Stuffing cache ----/538, done at 2023-06-19 21:30:19
2023-06-19 21:30:19,362 INFO     pid:18412 nb:021:run Finished: p2_ct_project.p2ch14.prepcache_luna_end_to_end.LunaPrepCacheApp.(['--num-workers=8', '--datasetdir=E:/Luna16']).main()


In [22]:
# ch14_epochs = 10
# run('p2_ct_preject.p2ch14.training_luna_end_to_end.ClassificationTrainingApp', 
#     f'--datasetdir={datasetdir}',
#     f'--epochs={ch14_epochs}', 
#     'nodule-nonnodule',
#     )

ファインチューニング (only head_linear)

In [23]:
# model_weight_path = 'models/'
# run('p2_ct_project.p2ch14.training_luna_end_to_end.ClassificationTrainingApp',
#     f'--dateasetdir={datasetdir}',
#     f'--epochs={ch14_epochs}',
#     '--malignant',
#     '--dataset=MalignantLunaDataset',
#     f'finetune={model_weight_path}',
#     'finetune-head',
#     )

ファインチューニング (head_linear and last conv)

In [24]:
# run('p2_ct_project.p2ch14.training_luna_end_to_end.ClassificationTrainingApp',
#     f'--dateasetdir={datasetdir}',
#     f'--epochs={ch14_epochs}',
#     '--malignant',
#     '--dataset=MalignantLunaDataset',
#     f'finetune={model_weight_path}',
#     'finetune-depth2',
# )

EndtoEndな結節検出システム

In [25]:
# run('p2_ct_project.p2ch14.nodule_analysis.NoduleAnalysisApp',
#     f'--datasetdir={datasetdir}',
#     '--run-validation',
#     )

EndtoEndな悪性腫瘍検出システム

In [26]:
# run('p2_ct_project.p2ch14.nodule_analysis.NoduleAnalysisApp',
#     f'--datasetdir={datasetdir}',
#     '--run-validation',
#     '--malignancy-path',
#     )
